In [ ]:
parse_dates = [f"time{i}" for i in range (1, 11)]

train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=parse_dates)
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=parse_dates)

train_df.head()

In [1]:
y = train_df['target']

NameError: name 'train_df' is not defined

In [ ]:
def get_sitestr(df=train_df):
    site_str = df['site1'].apply(lambda s: '' if np.isnan(s) else str(int(s)))
    for i in range(2, 11):
        site_str += " " + df[f"site{i}"].apply(lambda s: '' if np.isnan(s) else str(int(s)))
    return site_str.str.strip()
    
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=100000, lowercase=False)
train_sites = tfidf.fit_transform(get_sitestr(train_df))
test_sites = tfidf.transform(get_sitestr(test_df))
train_sites.shape, test_sites.shape

In [ ]:
def get_time(df=train_df):
    hour = df.time1.apply(lambda ts: ts.hour)
    time_features = [hour.between(i, i) for i in range(24)]
    weekdays = [df.time1.apply(lambda ts: ts.weekday()).between(i, i) for i in range(7)]
    yearmon = df.time1.apply(lambda ts: ts.year * 12 + ts.month)
    time_features += weekdays
    time_features.append(yearmon)
    return pd.concat(time_features, axis=1)

train_time = get_time(train_df)
test_time = get_time(test_df)

train_time.shape, test_time.shape

In [ ]:
time_scaler = StandardScaler()
train_time_scaled = time_scaler.fit_transform(train_time.values)
test_time_scaled = time_scaler.transform(test_time.values)
train_time_scaled.shape, test_time_scaled.shape

In [ ]:
X_train = hstack([train_sites, train_time_scaled])
X_test = hstack([test_sites, test_time_scaled])
X_train.shape, X_test.shape

In [ ]:
%%time
cv = LogisticRegressionCV(
    Cs=np.logspace(-5, 5, 20), cv=5,
    penalty='l2', scoring='roc_auc',
    random_state=RANDOM, n_jobs=N_JOBS)
cv.fit(X_train, y)

In [ ]:
cv.scores_[1].mean(axis=0).max()

In [ ]:
test_pred = cv.predict_proba(X_test)[:,1]

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file(test_pred, "assignment6_alice_submission.csv")